In [2]:
import utils.fetcher_utils as fetcher
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
# Get the data from the IMDb dataFrame
imdb_df = fetcher.aquireIMDbDataFrame()
imdb_df.head()

Reading data from ../resources\movie_metadata.csv


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0.0,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens ...,8,143,NaN,0.0,NaN,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [5]:
drop_columns = ['movie_imdb_link','aspect_ratio', 'plot_keywords']

imdb_df_filtered = imdb_df.drop(columns=drop_columns)
imdb_df_filtered = imdb_df_filtered.dropna(subset='title_year')


In [6]:
imdb_df_filtered = imdb_df_filtered[sorted(imdb_df_filtered.columns)]

In [7]:
imdb_df_filtered = imdb_df_filtered[(imdb_df_filtered['gross']>1_000) & 
                                    (imdb_df_filtered['budget']>1_000) & 
                                    (imdb_df_filtered['country'] == 'USA') &
                                    (imdb_df['title_year']>1994)].drop(columns='country')
imdb_df_filtered.head(5)

C:\Users\fagui\AppData\Local\Temp\ipykernel_17384\2842505215.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  imdb_df_filtered = imdb_df_filtered[(imdb_df_filtered['gross']>1_000) &


,actor_1_facebook_likes,actor_1_name,actor_2_facebook_likes,actor_2_name,actor_3_facebook_likes,actor_3_name,budget,cast_total_facebook_likes,color,content_rating,director_facebook_likes,director_name,duration,facenumber_in_poster,genres,gross,imdb_score,language,movie_facebook_likes,movie_title,num_critic_for_reviews,num_user_for_reviews,num_voted_users,title_year
0,1000.0,CCH Pounder,936.0,Joel David Moore,855.0,Wes Studi,237000000.0,4834,Color,PG-13,0.0,James Cameron,178.0,0.0,Action|Adventure|Fantasy|Sci-Fi,760505847.0,7.9,English,33000,Avatar,723.0,3054.0,886204,2009.0
1,40000.0,Johnny Depp,5000.0,Orlando Bloom,1000.0,Jack Davenport,300000000.0,48350,Color,PG-13,563.0,Gore Verbinski,169.0,0.0,Action|Adventure|Fantasy,309404152.0,7.1,English,0,Pirates of the Caribbean: At World's End,302.0,1238.0,471220,2007.0
3,27000.0,Tom Hardy,23000.0,Christian Bale,23000.0,Joseph Gordon-Levitt,250000000.0,106759,Color,PG-13,22000.0,Christopher Nolan,164.0,0.0,Action|Thriller,448130642.0,8.5,English,164000,The Dark Knight Rises,813.0,2701.0,1144337,2012.0
5,640.0,Daryl Sabara,632.0,Samantha Morton,530.0,Polly Walker,263700000.0,1873,Color,PG-13,475.0,Andrew Stanton,132.0,1.0,Action|Adventure|Sci-Fi,73058679.0,6.6,English,24000,John Carter,462.0,738.0,212204,2012.0
6,24000.0,J.K. Simmons,11000.0,James Franco,4000.0,Kirsten Dunst,258000000.0,46055,Color,PG-13,0.0,Sam Raimi,156.0,0.0,Action|Adventure|Romance,336530303.0,6.2,English,0,Spider-Man 3,392.0,1902.0,383056,2007.0


In [8]:
for column in imdb_df_filtered.columns:
    if(imdb_df_filtered[column].dtype == 'object'):
        print(column)
        imdb_df_filtered[column] = imdb_df_filtered[column].fillna('unknown')
        imdb_df_filtered[column] = imdb_df_filtered[column].apply(lambda x: x.strip())

actor_1_name
actor_2_name
actor_3_name
color
content_rating
director_name
genres
language
movie_title


In [9]:
imdb_df_filtered = imdb_df_filtered.fillna(-1)

In [10]:
imdb_df_filtered.isna().sum()

actor_1_facebook_likes       0
actor_1_name                 0
actor_2_facebook_likes       0
actor_2_name                 0
actor_3_facebook_likes       0
actor_3_name                 0
budget                       0
cast_total_facebook_likes    0
color                        0
content_rating               0
director_facebook_likes      0
director_name                0
duration                     0
facenumber_in_poster         0
genres                       0
gross                        0
imdb_score                   0
language                     0
movie_facebook_likes         0
movie_title                  0
num_critic_for_reviews       0
num_user_for_reviews         0
num_voted_users              0
title_year                   0
dtype: int64

In [11]:
# Concatenate with a comma separator
imdb_df_filtered['actors'] = imdb_df_filtered[['actor_1_name', 'actor_2_name', 'actor_3_name']].agg(list, axis=1)
imdb_df_filtered['actor_facebook_likes'] = imdb_df_filtered[['actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes']].agg(list, axis=1)

# join some columns and explode them 
imdb_df_filtered = imdb_df_filtered.explode(column=['actors', 'actor_facebook_likes']).reset_index(drop=True)


imdb_df_filtered

,actor_1_facebook_likes,actor_1_name,actor_2_facebook_likes,actor_2_name,actor_3_facebook_likes,actor_3_name,budget,cast_total_facebook_likes,color,content_rating,director_facebook_likes,director_name,duration,facenumber_in_poster,genres,gross,imdb_score,language,movie_facebook_likes,movie_title,num_critic_for_reviews,num_user_for_reviews,num_voted_users,title_year,actors,actor_facebook_likes
0,1000.0,CCH Pounder,936.0,Joel David Moore,855.0,Wes Studi,237000000.0,4834,Color,PG-13,0.0,James Cameron,178.0,0.0,Action|Adventure|Fantasy|Sci-Fi,760505847.0,7.9,English,33000,Avatar,723.0,3054.0,886204,2009.0,CCH Pounder,1000.0
1,1000.0,CCH Pounder,936.0,Joel David Moore,855.0,Wes Studi,237000000.0,4834,Color,PG-13,0.0,James Cameron,178.0,0.0,Action|Adventure|Fantasy|Sci-Fi,760505847.0,7.9,English,33000,Avatar,723.0,3054.0,886204,2009.0,Joel David Moore,936.0
2,1000.0,CCH Pounder,936.0,Joel David Moore,855.0,Wes Studi,237000000.0,4834,Color,PG-13,0.0,James Cameron,178.0,0.0,Action|Adventure|Fantasy|Sci-Fi,760505847.0,7.9,English,33000,Avatar,723.0,3054.0,886204,2009.0,Wes Studi,855.0
3,40000.0,Johnny Depp,5000.0,Orlando Bloom,1000.0,Jack Davenport,300000000.0,48350,Color,PG-13,563.0,Gore Verbinski,169.0,0.0,Action|Adventure|Fantasy,309404152.0,7.1,English,0,Pirates of the Caribbean: At World's End,302.0,1238.0,471220,2007.0,Johnny Depp,40000.0
4,40000.0,Johnny Depp,5000.0,Orlando Bloom,1000.0,Jack Davenport,300000000.0,48350,Color,PG-13,563.0,Gore Verbinski,169.0,0.0,Action|Adventure|Fantasy,309404152.0,7.1,English,0,Pirates of the Caribbean: At World's End,302.0,1238.0,471220,2007.0,Orlando Bloom,5000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7981,296.0,Kerry Bishé,205.0,Caitlin FitzGerald,133.0,Daniella Pineda,9000.0,690,Color,Not Rated,0.0,Edward Burns,95.0,1.0,Comedy|Drama,4584.0,6.4,English,413,Newlyweds,14.0,14.0,1338,2011.0,Caitlin FitzGerald,205.0
7982,296.0,Kerry Bishé,205.0,Caitlin FitzGerald,133.0,Daniella Pineda,9000.0,690,Color,Not Rated,0.0,Edward Burns,95.0,1.0,Comedy|Drama,4584.0,6.4,English,413,Newlyweds,14.0,14.0,1338,2011.0,Daniella Pineda,133.0
7983,86.0,John August,23.0,Brian Herzlinger,16.0,Jon Gunn,1100.0,163,Color,PG,16.0,Jon Gunn,90.0,0.0,Documentary,85222.0,6.6,English,456,My Date with Drew,43.0,84.0,4285,2004.0,John August,86.0
7984,86.0,John August,23.0,Brian Herzlinger,16.0,Jon Gunn,1100.0,163,Color,PG,16.0,Jon Gunn,90.0,0.0,Documentary,85222.0,6.6,English,456,My Date with Drew,43.0,84.0,4285,2004.0,Brian Herzlinger,23.0


In [12]:
# Calculate the frequency of each category
frequency = imdb_df_filtered['actors'].value_counts()

# Map the frequency back to the original dataframe
imdb_df_filtered['actors_Encoded'] = imdb_df_filtered['actors'].map(frequency)


# Calculate the frequency of each category
frequency = imdb_df_filtered['director_name'].value_counts()

# Map the frequency back to the original dataframe
imdb_df_filtered['directors_Encoded'] = imdb_df_filtered['director_name'].map(frequency)
imdb_df_filtered

,actor_1_facebook_likes,actor_1_name,actor_2_facebook_likes,actor_2_name,actor_3_facebook_likes,actor_3_name,budget,cast_total_facebook_likes,color,content_rating,director_facebook_likes,director_name,duration,facenumber_in_poster,genres,gross,imdb_score,language,movie_facebook_likes,movie_title,num_critic_for_reviews,num_user_for_reviews,num_voted_users,title_year,actors,actor_facebook_likes,actors_Encoded,directors_Encoded
0,1000.0,CCH Pounder,936.0,Joel David Moore,855.0,Wes Studi,237000000.0,4834,Color,PG-13,0.0,James Cameron,178.0,0.0,Action|Adventure|Fantasy|Sci-Fi,760505847.0,7.9,English,33000,Avatar,723.0,3054.0,886204,2009.0,CCH Pounder,1000.0,5,6
1,1000.0,CCH Pounder,936.0,Joel David Moore,855.0,Wes Studi,237000000.0,4834,Color,PG-13,0.0,James Cameron,178.0,0.0,Action|Adventure|Fantasy|Sci-Fi,760505847.0,7.9,English,33000,Avatar,723.0,3054.0,886204,2009.0,Joel David Moore,936.0,5,6
2,1000.0,CCH Pounder,936.0,Joel David Moore,855.0,Wes Studi,237000000.0,4834,Color,PG-13,0.0,James Cameron,178.0,0.0,Action|Adventure|Fantasy|Sci-Fi,760505847.0,7.9,English,33000,Avatar,723.0,3054.0,886204,2009.0,Wes Studi,855.0,5,6
3,40000.0,Johnny Depp,5000.0,Orlando Bloom,1000.0,Jack Davenport,300000000.0,48350,Color,PG-13,563.0,Gore Verbinski,169.0,0.0,Action|Adventure|Fantasy,309404152.0,7.1,English,0,Pirates of the Caribbean: At World's End,302.0,1238.0,471220,2007.0,Johnny Depp,40000.0,28,21
4,40000.0,Johnny Depp,5000.0,Orlando Bloom,1000.0,Jack Davenport,300000000.0,48350,Color,PG-13,563.0,Gore Verbinski,169.0,0.0,Action|Adventure|Fantasy,309404152.0,7.1,English,0,Pirates of the Caribbean: At World's End,302.0,1238.0,471220,2007.0,Orlando Bloom,5000.0,8,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7981,296.0,Kerry Bishé,205.0,Caitlin FitzGerald,133.0,Daniella Pineda,9000.0,690,Color,Not Rated,0.0,Edward Burns,95.0,1.0,Comedy|Drama,4584.0,6.4,English,413,Newlyweds,14.0,14.0,1338,2011.0,Caitlin FitzGerald,205.0,1,9
7982,296.0,Kerry Bishé,205.0,Caitlin FitzGerald,133.0,Daniella Pineda,9000.0,690,Color,Not Rated,0.0,Edward Burns,95.0,1.0,Comedy|Drama,4584.0,6.4,English,413,Newlyweds,14.0,14.0,1338,2011.0,Daniella Pineda,133.0,1,9
7983,86.0,John August,23.0,Brian Herzlinger,16.0,Jon Gunn,1100.0,163,Color,PG,16.0,Jon Gunn,90.0,0.0,Documentary,85222.0,6.6,English,456,My Date with Drew,43.0,84.0,4285,2004.0,John August,86.0,1,9
7984,86.0,John August,23.0,Brian Herzlinger,16.0,Jon Gunn,1100.0,163,Color,PG,16.0,Jon Gunn,90.0,0.0,Documentary,85222.0,6.6,English,456,My Date with Drew,43.0,84.0,4285,2004.0,Brian Herzlinger,23.0,1,9


In [13]:
processed_df = imdb_df_filtered

In [14]:
X = processed_df.drop(columns=['imdb_score'])
y = processed_df['imdb_score']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [16]:
dummy_columns = ['content_rating', 'color', 'language']
for col in dummy_columns:
    print(col)
    column_dummies = pd.get_dummies(X_train[col], prefix=col, prefix_sep='_', dtype='int') 
    X_train = pd.concat([X_train, column_dummies], axis=1)
    X_train.drop(columns=[col], inplace=True)
X_train

content_rating
color
language


,actor_1_facebook_likes,actor_1_name,actor_2_facebook_likes,actor_2_name,actor_3_facebook_likes,actor_3_name,budget,cast_total_facebook_likes,director_facebook_likes,director_name,duration,facenumber_in_poster,genres,gross,movie_facebook_likes,movie_title,num_critic_for_reviews,num_user_for_reviews,num_voted_users,title_year,actors,actor_facebook_likes,actors_Encoded,directors_Encoded,content_rating_G,content_rating_NC-17,content_rating_Not Rated,content_rating_PG,content_rating_PG-13,content_rating_R,content_rating_Unrated,content_rating_unknown,color_Black and White,color_Color,color_unknown,language_Bosnian,language_Dari,language_English,language_Filipino,language_Hindi,language_Japanese,language_Maya,language_Spanish,language_Vietnamese,language_unknown
2885,3000.0,Brendan Fraser,190.0,Paul Scheer,91.0,Jonathan Morgan Heit,40000000.0,3361,7.0,Cal Brunker,89.0,2.0,Adventure|Animation|Comedy|Family|Sci-Fi,57011847.0,0,Escape from Planet Earth,83.0,44.0,20615,2013.0,Jonathan Morgan Heit,91.0,1,3,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
5170,1000.0,Harold Perrineau,886.0,Sanaa Lathan,849.0,Eddie Cibrian,17000000.0,5306,92.0,Malcolm D. Lee,123.0,5.0,Comedy|Drama,70492685.0,7000,The Best Man Holiday,56.0,64.0,11600,2013.0,Sanaa Lathan,886.0,8,15,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
640,12000.0,Dwayne Johnson,2000.0,Ioan Gruffudd,884.0,Archie Panjabi,110000000.0,16718,62.0,Brad Peyton,114.0,0.0,Action|Adventure|Drama|Thriller,155181732.0,52000,San Andreas,358.0,499.0,147497,2015.0,Ioan Gruffudd,2000.0,5,9,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
2432,1000.0,Christina Milian,923.0,Vanessa Ferlito,730.0,Kelli Garner,40000000.0,6171,65.0,Stephen Herek,100.0,5.0,Action|Comedy,19118247.0,779,Man of the House,59.0,94.0,19829,2005.0,Kelli Garner,730.0,5,15,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
4252,11000.0,Chris Evans,673.0,Jason Patric,619.0,Óscar Jaenada,25000000.0,12841,41.0,Sylvain White,97.0,1.0,Action|Crime|Drama|Mystery|Thriller,23527955.0,0,The Losers,213.0,165.0,74691,2010.0,Jason Patric,673.0,4,6,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5226,21000.0,Robert Downey Jr.,3000.0,Ty Burrell,260.0,Matt Servitto,16800000.0,25173,30.0,Steven Shainberg,122.0,2.0,Biography|Drama|Romance,220914.0,0,Fur: An Imaginary Portrait of Diane Arbus,105.0,97.0,13239,2006.0,Robert Downey Jr.,21000.0,27,6,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
5390,20000.0,Natalie Portman,1000.0,James Frain,944.0,Stockard Channing,15000000.0,22383,18.0,Matt Williams,120.0,2.0,Comedy|Drama|Romance,33771174.0,0,Where the Heart Is,78.0,240.0,27044,2000.0,Stockard Channing,944.0,8,3,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
860,16000.0,Christopher Lee,5000.0,Orlando Bloom,857.0,Billy Boyd,94000000.0,23052,0.0,Peter Jackson,172.0,1.0,Action|Adventure|Drama|Fantasy,340478898.0,10000,The Lord of the Rings: The Two Towers,294.0,2417.0,1100446,2002.0,Billy Boyd,857.0,2,18,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
7603,443.0,David Hyde Pierce,298.0,Clayne Crawford,271.0,Nathaniel Parker,500000.0,1452,11.0,Nick Tomnay,93.0,1.0,Comedy|Crime|Thriller,48430.0,0,The Perfect Host,112.0,73.0,20176,2010.0,Clayne Crawford,298.0,2,3,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0


In [23]:
# X_train.info()

In [22]:
# X_train.describe()

In [19]:
drop_columns = ['actor_1_facebook_likes', 'actor_1_name', 'actor_2_facebook_likes','actor_2_name', 'actor_3_facebook_likes', 'actor_3_name', 'director_name', 'actors', 'movie_title']

In [20]:
X_train = X_train.drop(columns=drop_columns).reset_index(drop=True)
X_train

,budget,cast_total_facebook_likes,director_facebook_likes,duration,facenumber_in_poster,genres,gross,movie_facebook_likes,num_critic_for_reviews,num_user_for_reviews,num_voted_users,title_year,actor_facebook_likes,actors_Encoded,directors_Encoded,content_rating_G,content_rating_NC-17,content_rating_Not Rated,content_rating_PG,content_rating_PG-13,content_rating_R,content_rating_Unrated,content_rating_unknown,color_Black and White,color_Color,color_unknown,language_Bosnian,language_Dari,language_English,language_Filipino,language_Hindi,language_Japanese,language_Maya,language_Spanish,language_Vietnamese,language_unknown
0,40000000.0,3361,7.0,89.0,2.0,Adventure|Animation|Comedy|Family|Sci-Fi,57011847.0,0,83.0,44.0,20615,2013.0,91.0,1,3,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
1,17000000.0,5306,92.0,123.0,5.0,Comedy|Drama,70492685.0,7000,56.0,64.0,11600,2013.0,886.0,8,15,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
2,110000000.0,16718,62.0,114.0,0.0,Action|Adventure|Drama|Thriller,155181732.0,52000,358.0,499.0,147497,2015.0,2000.0,5,9,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
3,40000000.0,6171,65.0,100.0,5.0,Action|Comedy,19118247.0,779,59.0,94.0,19829,2005.0,730.0,5,15,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
4,25000000.0,12841,41.0,97.0,1.0,Action|Crime|Drama|Mystery|Thriller,23527955.0,0,213.0,165.0,74691,2010.0,673.0,4,6,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5984,16800000.0,25173,30.0,122.0,2.0,Biography|Drama|Romance,220914.0,0,105.0,97.0,13239,2006.0,21000.0,27,6,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
5985,15000000.0,22383,18.0,120.0,2.0,Comedy|Drama|Romance,33771174.0,0,78.0,240.0,27044,2000.0,944.0,8,3,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
5986,94000000.0,23052,0.0,172.0,1.0,Action|Adventure|Drama|Fantasy,340478898.0,10000,294.0,2417.0,1100446,2002.0,857.0,2,18,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
5987,500000.0,1452,11.0,93.0,1.0,Comedy|Crime|Thriller,48430.0,0,112.0,73.0,20176,2010.0,298.0,2,3,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0


In [21]:
X_train['genres'] = X_train['genres'].apply(lambda x: x.split('|'))
genre_dummies = X_train['genres'].explode().str.get_dummies().groupby(level=0).max()
X_train = pd.concat([X_train.drop(columns=['genres']), genre_dummies], axis=1)
X_train

,budget,cast_total_facebook_likes,director_facebook_likes,duration,facenumber_in_poster,gross,movie_facebook_likes,num_critic_for_reviews,num_user_for_reviews,num_voted_users,title_year,actor_facebook_likes,actors_Encoded,directors_Encoded,content_rating_G,content_rating_NC-17,content_rating_Not Rated,content_rating_PG,content_rating_PG-13,content_rating_R,content_rating_Unrated,content_rating_unknown,color_Black and White,color_Color,color_unknown,language_Bosnian,language_Dari,language_English,language_Filipino,language_Hindi,language_Japanese,language_Maya,language_Spanish,language_Vietnamese,language_unknown,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,40000000.0,3361,7.0,89.0,2.0,57011847.0,0,83.0,44.0,20615,2013.0,91.0,1,3,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
1,17000000.0,5306,92.0,123.0,5.0,70492685.0,7000,56.0,64.0,11600,2013.0,886.0,8,15,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,110000000.0,16718,62.0,114.0,0.0,155181732.0,52000,358.0,499.0,147497,2015.0,2000.0,5,9,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
3,40000000.0,6171,65.0,100.0,5.0,19118247.0,779,59.0,94.0,19829,2005.0,730.0,5,15,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,25000000.0,12841,41.0,97.0,1.0,23527955.0,0,213.0,165.0,74691,2010.0,673.0,4,6,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5984,16800000.0,25173,30.0,122.0,2.0,220914.0,0,105.0,97.0,13239,2006.0,21000.0,27,6,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
5985,15000000.0,22383,18.0,120.0,2.0,33771174.0,0,78.0,240.0,27044,2000.0,944.0,8,3,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
5986,94000000.0,23052,0.0,172.0,1.0,340478898.0,10000,294.0,2417.0,1100446,2002.0,857.0,2,18,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
5987,500000.0,1452,11.0,93.0,1.0,48430.0,0,112.0,73.0,20176,2010.0,298.0,2,3,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
imdb_df_filtered['actors_Encoded'].value_counts()

# Label Encoder

In [ ]:
##### do we encode before or after splitting data
### need to fix 

# label_encoder = LabelEncoder()
# object_columns = imdb_df_filtered.select_dtypes(include='object').columns

# for column in object_columns:
#     encoded_column = f"{column}_encoded"  
#     imdb_df_filtered[encoded_column] = label_encoder.fit_transform(imdb_df_filtered[column])

In [ ]:
# imdb_df_filtered['director_name'].value_counts().head(50)

In [ ]:
# imdb_df_numeric = imdb_df_filtered.select_dtypes(include='number')
# imdb_df_numeric

In [ ]:
# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 7))

# Plot the histogram on the first subplot
ax1.hist(imdb_df_filtered['imdb_score'], bins=10, edgecolor='black')
ax1.set_title('Histogram')
ax1.set_xlabel('IMDB score')
ax1.set_ylabel('Frequency')

# Plot the boxplot on the second subplot
ax2.boxplot(imdb_df_filtered['imdb_score'])
ax2.set_title('Boxplot')
ax2.set_ylabel('IMDB score')

# Show the figure
plt.show()

In [ ]:
# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Plot the histogram on the first subplot
ax1.hist(imdb_df_filtered['gross'], bins=100, edgecolor='black')
ax1.set_title('Histogram')
ax1.set_xlabel('Gross')
ax1.set_ylabel('Frequency')

# Plot the histogram on the first subplot
ax2.hist(imdb_df_filtered['budget'], bins=30, edgecolor='black')
ax2.set_title('Histogram')
ax2.set_xlabel('Budget')
ax2.set_ylabel('Frequency')

# Show the figure
plt.show()

In [ ]:
# gross_na_rows = imdb_df_filtered.loc[imdb_df_filtered['gross'].isna()]
# movie_list = gross_na_rows['movie_title'].apply(lambda x: x.strip('\xa0')).unique()


# million_df = tmdb_df_filtered.loc[tmdb_df_filtered['original_title'].isin(movie_list)].sort_values(by='original_title')
# million_df

# million_df = tmdb_df_filtered[tmdb_df_filtered['original_title'].str.lower().isin([x.lower() for x in movie_list])]
# million_df


In [ ]:
# Sample data
data = {
    'Category': ['A', 'B', 'A', 'C', 'B', 'C', 'A', 'D', 'E', 'C'],
    'Target': [1, 0, 1, 0, 1, 0, 1, 1, 0, 0]
}
df = pd.DataFrame(data)

# Calculate the mean of the target for each category
target_mean = df.groupby('Category')['Target'].mean()

# Map the mean back to the original dataframe
df['Category_Encoded'] = df['Category'].map(target_mean)

In [ ]:
len(set(imdb_df_filtered.groupby('actor_1_name')['imdb_score'].mean().values))

In [ ]:
target_mean

In [ ]:
df